# The Data Science Process

## Professional Ethics

Data Science and Machine Learning (and more broadly Computer Science) have great power. The techniques we will learn in this class are leading to algorithms giving computers the ability to make decisions based on a wealth of information. Facial recongition, credit and loan decisions, insurance pricing, are all examples of ways in which the tools we are learning in this class are being used.

It is not then surprising that with these tools comes some questions of professional ethics. Today we will discuss a few things to keep in mind. The first step in our Data Science process is to consider the ethical implications of the data collection and question we are setting out on.

#### Models Inherit Biases from the Data

If your model is built upon data from a system with biases (credit data for example), it is likely your model will inherit those biases. It is not enough to say that the process of building the model does not introduce bias, we also must be aware of whether the data we build that model from has biases. 

#### Keep in Mind the Consequences of Errors

As you build your model keep in mind the consequences of the errors in your model in the event it goes into production. These should guide you in the level of accuracy and the types and proportions of errors you can tolerate. An algorithm estimating the age of a trea from some features versus one being used to determine a likliehood that a prisoner will reoffend if paroled have much difference tolerances of error. 

#### Research on Human Subjects

There are restrictions on how data can be collected and used when the research is being conducted on human subjects. At federally funded institutions, proposals to do research on human subjects must be reviewed by institutional review boards. They are looking for among other things: How the privacy of subjects is being protected, how the risk of harm to subjects is being minimized, and how subjects are informed of their rights. Of particular importance is protecting vulnerable subjects (minors and prisoners for example).

This review must be done before data collection can be begin (typically).

There are similar reviews that must occur for Research on Animals.

#### Commitments Made about Data and Data Analysis

In obtaining permission from an Institutional Review Board, or permission from a company or individual owning data you wish to use, you or your company might make some promises about how that data will be used or how it will be protected. For example, in obtaining permission from Twitter to access their API you will be required to promise that you will not seek to obtain information about the political affiliation of an individual user, or that you will not publish content from Twitter without their permission. You need to honor promises that you or your employer have made regarding data and its protection. This is essential to ensuring trust in the profession, and to protecting vulnerable populations and individuals privacy, and it is essential to protecting yourself. When in doubt you should consult a lawyer before violating a commitment you have made. 

#### Follow the Error Control Process

It is essential that when you finish the development of your model that you have as good an understanding of the accuracy of your model as you can. For example, the accuracy of your model, and the consequences for error may be one of the ethical issues you have dealt with. However even if your model is realtively innocous, you need to be able to understand its utility and pay off were it to be used in production. 

It is essnetial in your data science work you follow a process that will result in an accurate esitmate of the accuracy of your model. 

## Controlling for Error

### Error in Regression Problems

We will skip the *gathering data* step of the process for now. We have some data available in a CSV file. Before we dive into an exploratory data analysis and developing a model, we should make a plan for how we will estimate the error of our model. Or even what we mean by the error of our model.

Today, because I want us to be sure we understand the sources of error in our analysis we will build the dataset in Python in a way where we actually know the *correct model*. You should be thinking about the House Pricing problem or one of the other regression problems we have talked about.

Suppose we have a predictor feature $x$ and a resulting value $y$ that have a linear relationship, however because we cannot possible account for all of the variation there is some *Irreducible Error*. That is when we build our sample, there is a random variable that comes along with it.


In [2]:
# Using the usual packages, plus scikitlearns LinearRegression tool; and numpy.random.

import numpy as np
import numpy.random as rn
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
# Building a sample with a linear relationship y = 10 x + 7 + E; where E is a random normally 
# distributed error term with mean 0 and variance 1.

x = 